In [35]:
import pandas as pd
import numpy as np

In [36]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [37]:
df_last_15_days = df.loc[df['timestamp'] > pd.to_datetime('2018-05-20')]
df_last_15_days = df_last_15_days.loc[(df_last_15_days['event']=='checkout') | 
                                          (df_last_15_days['event']=='viewed product'), :]

In [38]:
grouped = df_last_15_days.groupby(['person', 'model']).agg({'timestamp':'count'})

In [39]:
grouped.reset_index(inplace=True)
grouped.columns = ['person', 'model', 'cantidadMaximaDeTelefonoVisto']

In [40]:
grouped = grouped.groupby(['person']).agg({'cantidadMaximaDeTelefonoVisto': 'max'})

In [43]:
personas_sin_repetir = df.drop_duplicates(subset="person", keep='first')
personas_sin_repetir = personas_sin_repetir.loc[:,"person"]
personas_sin_repetir = personas_sin_repetir.to_frame()
personas_sin_repetir.set_index("person",inplace=True)

In [47]:
df_conversion = df.loc[df['timestamp'] > pd.to_datetime('2018-05-20')]

In [50]:
df_conversion = df_conversion.loc[(df_conversion['event']=='conversion'), :]
df_conversion = df_conversion.drop_duplicates(subset="person", keep='first')

In [56]:
df_conversion = df_conversion.loc[:,["person"]]
df_conversion["tieneConversion"] = True
df_conversion.set_index("person",inplace=True)

In [59]:
df_final = personas_sin_repetir.join(grouped,on="person",how="left")
df_final = df_final.join(df_conversion,on="person",how="left")

In [62]:
import numpy as np
df_final["cantidadMaximaDeTelefonoVisto"] = np.where(df_final["tieneConversion"]==True,0,df_final["cantidadMaximaDeTelefonoVisto"] )

In [66]:
df_final = df_final.loc[:,["cantidadMaximaDeTelefonoVisto"]]
df_final.fillna(0,inplace=True)

In [67]:
df_final

,cantidadMaximaDeTelefonoVisto
person,
4886f805,0.0
ad93850f,0.0
0297fc1e,48.0
2d681dd8,6.0
cccea85e,67.0
4c8a8b93,15.0
1b9f7cf6,0.0
29ebb414,0.0
de8fe91b,4.0


In [68]:
df_final.to_csv('../data/cant_max_eventos_sobre_un_mismo_telefono_ult_quincena.csv')